In [1]:
import pandas as pd
import os
import numpy as np
import time
from datetime import datetime
to_datetime = lambda d: pd.to_datetime(d, format='%d/%m/%Y')

In [2]:
dir = 'C:\\MegaBolsa\\cotacoes\\dados\\b3\\cotacoes_diarias\\'

# arquivo = 'QUAL3.sa.csv'

## Inicio
inicio = time.perf_counter()
micos = pd.DataFrame()
x = y = 0

## Varrer os arquivos no diretório
for arquivo in os.listdir(dir):
    
    ## Lê os arquivos
    cotacoes = pd.read_csv(dir + arquivo, parse_dates=['date'], date_parser=to_datetime).sort_values(by='date', ascending=True)

    ## Cálculos
    cotacoes['simbolo'] = arquivo[0:-7]
    cotacoes['vol_med'] = cotacoes['vol'].rolling(window=100).mean()      ## Média Móvel do Volume
    cotacoes['var_vol'] = cotacoes['vol']/cotacoes['vol'].shift(1)        ## Variação do Volume no dia
    cotacoes['var_med'] = cotacoes['vol']/cotacoes['vol_med']             ## Variação do Volume em relação a Média Móvel
    cotacoes['saida'] = cotacoes['open'].shift(-1)                        ## Saída da Estratégia
    cotacoes['lucro'] = cotacoes['saida'] - cotacoes['close']             ## Lucro da Estrategia
    cotacoes['var_lucro'] = (cotacoes['lucro']/cotacoes['close']) * 100   ## Variação % do Lucro
    cotacoes['var_close'] = (cotacoes['saida'] / cotacoes['saida'].shift(1) - 1) * 100 ## Variação % do Fechamento
    
    ## Seleção
    selecao = (cotacoes['var_vol'] >= 10) | (cotacoes['var_med'] >= 10) ## Variação diária ou relativa a média > 10

    if len(micos) == 0:
        micos = cotacoes[selecao].copy()
    else:
        micos = micos.append(cotacoes[selecao])

    ## Contadores
    x += 1
    y += len(cotacoes)

micos.set_index(['simbolo','date'], inplace=True)

## Fim
fim = time.perf_counter()

Tempo:  37.501433585
695 simbolos, 952600 registros


In [12]:
print("Tempo: ", fim - inicio) 
print("{} simbolos, {} registros, {} selecionados".format(x, y, len(micos)))
micos.info()

Tempo:  37.501433585
695 simbolos, 952600 registros, 51759 selecionados
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 51759 entries, (ABCB4, 2007-10-22 00:00:00) to (XTED11, 2019-06-21 00:00:00)
Data columns (total 12 columns):
open         51759 non-null float64
high         51759 non-null float64
low          51759 non-null float64
close        51759 non-null float64
neg          51759 non-null object
vol          51759 non-null object
vol_med      47541 non-null float64
var_vol      51759 non-null float64
var_med      47541 non-null float64
saida        51736 non-null float64
lucro        51736 non-null float64
var_lucro    51736 non-null float64
dtypes: float64(10), object(2)
memory usage: 5.0+ MB


In [8]:
micos.head()

open   high    low  close  neg         vol  vol_med  \
simbolo date                                                               
ABCB4   2007-10-22  14.85  15.30  14.85  15.20   83  1140994600      NaN   
        2007-11-14  13.70  14.10  13.65  13.95  187  1102611700      NaN   
        2007-12-17  12.29  12.29  11.50  11.60   41   187024200      NaN   
        2008-01-04  11.45  11.45  10.50  10.85   23    93884600      NaN   
        2008-02-08  10.20  10.44  10.15  10.40   38   175347300      NaN   

                      var_vol  var_med  saida  lucro  var_lucro  
simbolo date                                                     
ABCB4   2007-10-22  13.230656      NaN  15.35   0.15   0.986842  
        2007-11-14  86.564216      NaN  14.00   0.05   0.358423  
        2007-12-17  16.045453      NaN  11.93   0.33   2.844828  
        2008-01-04  37.472899      NaN  10.80  -0.05  -0.460829  
        2008-02-08  11.094419      NaN  10.45   0.05   0.480769

In [10]:
micos.query("simbolo == 'QUAL3'").head()

open   high    low  close  neg         vol  vol_med  \
simbolo date                                                               
QUAL3   2011-07-08  14.61  14.75  14.40  14.75   20   106698600      NaN   
        2011-07-11  14.59  14.60  14.25  14.49   58  1380122900      NaN   
        2011-07-14  13.90  13.95  13.80  13.95   31   844844900      NaN   
        2011-07-22  14.40  14.75  14.40  14.75  202   454799500      NaN   
        2011-07-27  14.73  14.85  14.67  14.80   68  1769924200      NaN   

                       var_vol  var_med  saida  lucro  var_lucro  
simbolo date                                                      
QUAL3   2011-07-08   15.656664      NaN  14.59  -0.16  -1.084746  
        2011-07-11   12.934780      NaN  14.10  -0.39  -2.691511  
        2011-07-14   10.492739      NaN  13.90  -0.05  -0.358423  
        2011-07-22   19.181921      NaN  14.74  -0.01  -0.067797  
        2011-07-27  102.256334      NaN  14.59  -0.21  -1.418919

In [11]:
len(micos)

51759